In [16]:
import pandas as pd
import classifiers
import importlib
import os
import pickle
import itertools
from tqdm.notebook import tqdm, tnrange
import numpy as np
from igraph import Graph
from joblib import Parallel, delayed

# Functions

In [2]:
def check_dir(dir):
    """
    """
    if os.path.exists(dir) and os.path.isdir(dir):
        pass
    else:
        os.makedirs(dir)

# Directories

In [3]:
parent_dir = '../data/processed/'
modules_dir = parent_dir + 'modules/'
network_dir = parent_dir + 'networks/'

In [4]:
networks = ['apid_huri', 'string']
datasets = ['reactome', 'disgenet_sca', 'disgenet_conservative']
false_annotations = [False, True]
directories = {net: {} for net in networks}

for net in networks:
    for dir_ in ['graph', 'metrics', 'labels', 'models']:
        new_dir = network_dir + net + '/' + dir_ +'/'
        check_dir(new_dir)
        directories[net][dir_] = new_dir

# TEST CLIQUES

## Functions

In [2]:
def check_dir(dir):
    """
    """
    if os.path.exists(dir) and os.path.isdir(dir):
        pass
    else:
        os.makedirs(dir)

## Data

In [29]:
parent_dir = '../data/processed/'
modules_dir = parent_dir + 'modules/'
network_dir = parent_dir + 'networks/'
networks = ['apid_huri', 'string']
directories = {net: {} for net in networks}

for net in networks:
    for dir_ in ['graph', 'metrics', 'labels', 'models']:
        new_dir = network_dir + net + '/' + dir_ +'/'
        check_dir(new_dir)
        directories[net][dir_] = new_dir

In [30]:
metrics = pd.read_csv(directories['apid_huri']['metrics'] + 'reactome_rwr.csv', index_col=0)
metrics.head(2)

,R-HSA-481007,R-HSA-6798748,R-HSA-6800434,R-HSA-8952289,R-HSA-560473,R-HSA-560517,R-HSA-376419,R-HSA-382096,R-HSA-9013145,R-HSA-9018808,...,R-HSA-9629483,R-HSA-9629497,R-HSA-9670101,R-HSA-9670114,R-HSA-9670149,R-HSA-9698928,R-HSA-9710490,R-HSA-977224,R-HSA-6799179,R-HSA-6799196
SCRIB,0.000223,0.000155,0.000155,0.000158,0.000203,0.000203,0.000338,0.000334,0.000216,0.000225,...,0.000134,0.000134,0.000135,0.000134,0.000140,0.000134,0.000127,0.000126,0.000126,0.000126
ARHGEF7,0.000060,0.000065,0.000064,0.000056,0.000096,0.000096,0.000060,0.000059,0.000572,0.000279,...,0.000047,0.000047,0.000047,0.000047,0.000049,0.000046,0.000046,0.000046,0.000046,0.000046


In [31]:
labels = pd.read_csv(directories['apid_huri']['labels'] + 'reactome.csv', index_col=0)
labels.head(2)

,R-HSA-481007,R-HSA-6798748,R-HSA-6800434,R-HSA-8952289,R-HSA-560473,R-HSA-560517,R-HSA-376419,R-HSA-382096,R-HSA-9013145,R-HSA-9018808,...,R-HSA-9629483,R-HSA-9629497,R-HSA-9670101,R-HSA-9670114,R-HSA-9670149,R-HSA-9698928,R-HSA-9710490,R-HSA-977224,R-HSA-6799179,R-HSA-6799196
SCRIB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ARHGEF7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
import scipy.stats as stats

In [33]:
spearman = stats.spearmanr(metrics).statistic

In [8]:
correlation = np.copy(spearman)
correlation[correlation>0.95] = 1
correlation[correlation!=1] = 0
correlation

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.]])

In [9]:
g = Graph.Adjacency(correlation, mode="undirected")
g.vs['name'] = [i for i in metrics.columns.to_list()]

In [10]:
g.is_connected()

False

In [11]:
print(g.components())

Clustering with 428 elements and 22 clusters
[ 0] R-HSA-481007
[ 1] R-HSA-6798748, R-HSA-6800434, R-HSA-560473, R-HSA-560517, R-HSA-376419,
     R-HSA-382096, R-HSA-2288097, R-HSA-2294574, R-HSA-2294580, R-HSA-2294590,
     R-HSA-2294600, R-HSA-141409, R-HSA-141422, R-HSA-141431, R-HSA-141439,
     R-HSA-1638803, R-HSA-1638821, R-HSA-2467809, R-HSA-2467811,
     R-HSA-2468287, R-HSA-2484822, R-HSA-375302, R-HSA-5666129, R-HSA-5666160,
     R-HSA-5666169, R-HSA-606287, R-HSA-606289, R-HSA-9648114, R-HSA-3223200,
     R-HSA-3229089, R-HSA-3229102, R-HSA-156661, R-HSA-6782141, R-HSA-6782204,
     R-HSA-6782208, R-HSA-6782211, R-HSA-6782224, R-HSA-6782227, R-HSA-72127,
     R-HSA-72130, R-HSA-72139, R-HSA-72143, R-HSA-72160, R-HSA-983140,
     R-HSA-983147, R-HSA-983156, R-HSA-983157, R-HSA-75079, R-HSA-75081,
     R-HSA-75082, R-HSA-75083, R-HSA-72124, R-HSA-5682858, R-HSA-5683077,
     R-HSA-5683384, R-HSA-5683385, R-HSA-5683735, R-HSA-5683801,
     R-HSA-5684052, R-HSA-5684071, R-HSA-56

In [12]:
g.ecount()

18155

In [13]:
g.clique_number()

155

In [24]:
l_cliques = g.largest_cliques()

In [25]:
for i in l_cliques:
    print(len(l_cliques))

1


In [14]:
g.maximal_cliques(min=2)

[(264, 265),
 (2, 1),
 (79, 80),
 (79, 19),
 (80, 231),
 (427, 426),
 (229, 230),
 (262, 20, 263),
 (14, 15, 16),
 (134, 132, 133),
 (9, 8, 81),
 (192, 189, 191, 190),
 (12, 10, 13, 11),
 (141, 140, 224, 223),
 (8, 81, 221, 220, 131, 129),
 (223, 98, 224, 126, 125, 99),
 (61, 58, 60, 59, 149, 156, 153, 151, 150),
 (61, 58, 60, 59, 255, 254, 253, 251, 250, 104, 103, 102),
 (61, 58, 60, 152, 156, 149, 150, 151, 153),
 (6, 4, 195, 194, 193, 52, 51, 50, 48, 47, 5, 7),
 (6, 4, 195, 194, 193, 52, 51, 50, 48, 47, 5, 49),
 (215, 67, 217, 216, 214, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68),
 (345, 239, 344, 343, 342, 341, 340, 320, 319, 318, 317, 316, 242, 241, 240),
 (125, 42, 238, 237, 236, 235, 234, 233, 232, 126, 114, 99, 97, 98),
 (125, 42, 238, 237, 236, 235, 234, 233, 232, 126, 114, 99, 97, 124, 122, 117),
 (126,
  30,
  238,
  237,
  236,
  235,
  234,
  233,
  232,
  114,
  99,
  97,
  42,
  36,
  35,
  34,
  33,
  32,
  31,
  98),
 (126,
  30,
  238,
  237,
  236,
  235,
  234,
  233,
 

In [15]:
def find_cliques(graph):

    g = graph.copy()
    v_names = np.array(g.vs['name'])
    cliques = []

    while True:
        l_clique = np.array(g.largest_cliques()[0]) # array of ids (Int)
        
        #if max(g.largest_cliques()) == 1 and sum([len(c) for c in g.largest_cliques()]) == len(g.vs):
            
        cliques.append(v_names[l_clique]) # array of names (String)
        
        # remove nodes present in the largest clique
        rest = np.setdiff1d(np.arange(g.vcount()), l_clique)

        if len(rest) > 0: 
            g = g.induced_subgraph(rest)
            v_names = np.array(g.vs['name'])

        else:
            return cliques

In [20]:
c = find_cliques(g)
print(len(c))
c

38


[array(['R-HSA-1214188', 'R-HSA-2288097', 'R-HSA-977224', 'R-HSA-9710490',
        'R-HSA-9698928', 'R-HSA-9670149', 'R-HSA-9670114', 'R-HSA-9670101',
        'R-HSA-9629497', 'R-HSA-9629483', 'R-HSA-9629373', 'R-HSA-9629372',
        'R-HSA-9629216', 'R-HSA-9629195', 'R-HSA-9623096', 'R-HSA-9621073',
        'R-HSA-9614811', 'R-HSA-9604834', 'R-HSA-9604831', 'R-HSA-9604829',
        'R-HSA-9012000', 'R-HSA-9011983', 'R-HSA-9011966', 'R-HSA-9011952',
        'R-HSA-9011950', 'R-HSA-9011949', 'R-HSA-9009378', 'R-HSA-9009371',
        'R-HSA-9009065', 'R-HSA-8937118', 'R-HSA-8937113', 'R-HSA-8937022',
        'R-HSA-8936989', 'R-HSA-8936608', 'R-HSA-8936599', 'R-HSA-8936584',
        'R-HSA-8935730', 'R-HSA-8865505', 'R-HSA-8865498', 'R-HSA-8865491',
        'R-HSA-73718', 'R-HSA-5625883', 'R-HSA-5625870', 'R-HSA-5625849',
        'R-HSA-5625848', 'R-HSA-5625797', 'R-HSA-5625796', 'R-HSA-5625784',
        'R-HSA-5625774', 'R-HSA-5617650', 'R-HSA-5617642', 'R-HSA-5617462',
        'R-HSA-

In [21]:
rep = [i for a in c for i in a]
len(rep)

428

In [22]:
for clique in c:
    print(len(clique))

155
68
45
42
15
15
12
10
6
6
5
4
4
4
3
3
3
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [19]:
print('# of nodes:', len(c.keys()))
print('max # of cliques:', max(c.values()))
print('# of nodes (max cliques):', len([k for k, v in c.items() if v == max(c.values())]))
print('# of nodes (1 clique):', len([k for k, v in c.items() if v == 1]))
print('median cliques:', np.median(list(c.values())))

AttributeError: 'list' object has no attribute 'keys'

# GAP-MINE

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [24]:
importlib.reload(classifiers)
network_label='apid_huri'
dataset_label='reactome'
metrics = pd.read_csv(directories[network_label]['metrics'] + f"{dataset_label}_rwr.csv", header=0, index_col=0)
labels = pd.read_csv(directories[network_label]['labels'] + f"{dataset_label}.csv", header=0, index_col=0)
X = metrics.to_numpy()
y = labels.to_numpy(dtype='int')
module_labels = metrics.columns.to_list()

In [38]:
importlib.reload(classifiers)
topfeatures = []
for module in tnrange(len(module_labels)):
    y_mod = y[:, module]
      
    X_train, X_test, y_train, y_test = train_test_split(
            X, y_mod, train_size=0.8, stratify=y_mod, random_state=0
            )

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    
    features, _ = classifiers.feature_selection(X_train, y_train, module)
    topfeatures.append(features)

In [ ]:
importlib.reload(classifiers)
for dataset_label, network_label, fa in itertools.product(datasets, networks, false_annotations):
    
    ############### load data #############################
    if fa:
        fa_label = '_fa_'
        graph = Graph.Read_GML(directories[network_label]['graph'] + "graph.gml")
        shortest_paths = np.array(graph.distances())

    else:
        fa_label = '_'
        graph = None
        shortest_paths = None
        
    metrics = pd.read_csv(directories[network_label]['metrics'] + f"{dataset_label}_rwr.csv", header=0, index_col=0)

    labels = pd.read_csv(directories[network_label]['labels'] + f"{dataset_label}.csv", header=0, index_col=0)   
    
    ####################### Prepare data #######################################################

    module_labels = metrics.columns.to_list()

    X = metrics.to_numpy()
    y = labels.to_numpy(dtype='int')
    
    ###################### Compute GAP-MINE ######################################################
    results = Parallel(n_jobs=12)(
        delayed(classifiers.gapmine)(
            X, y, module, train_size=0.8, beta=1, false_annotations=fa, shortest_paths=shortest_paths, graph=graph, random_state=42
            ) for module in tqdm(range(len(module_labels)))
        )
    
    cv_results = pd.DataFrame([i[0] for i in results])
    cv_results['target_module'] = module_labels
    clf_results = pd.DataFrame([i[1] for i in results])
    clf_results['target_module'] = module_labels
    clf_models = [i[2] for i in results]

    ###################### Save Results ###########################################################
    gapmine_dir = directories[network_label]['models'] + 'gapmine/'
    check_dir(gapmine_dir)
    clf_results.to_csv(gapmine_dir + f'{dataset_label}{fa_label}results.csv', index=False)
    cv_results.to_csv(gapmine_dir + f'{dataset_label}{fa_label}cv.csv', index=False)

    with open(gapmine_dir + f"{dataset_label}{fa_label}models.pckl", "wb") as f:
        for model in clf_models:
            pickle.dump(model, f)

# Baseline Model

In [ ]:
metrics_dir = "../../data/processed/metrics/"
labels_dir = "../../data/processed/"
fs_dir = "../../data/processed/fs/"

In [53]:
directory = '../../models/baseline_model/'

In [ ]:
networks = ['apid_huri']
datasets = ['reactome', 'disgenet_sca', 'disgenet_conservative']
false_annotations = [False, True]

graph = Graph.Read_GML("../../data/processed/apid_huri_graph")

In [ ]:
# compute shortest paths
shortest_paths = graph.distances()
shortest_paths = np.array(shortest_paths)

In [56]:
importlib.reload(classifiers)
for dataset_label, network_label, fa in itertools.product(datasets, networks, false_annotations):
    
    if os.path.exists(directory) and os.path.isdir(directory):
        pass
    else:
        os.makedirs(directory)

    ############### load data #############################
    if fa:
        fa_label = '_fa_'
        g=graph
        sp=shortest_paths

    else:
        fa_label = '_'
        g=None
        sp=None
        
    metrics = pd.read_csv(metrics_dir + f"{dataset_label}_{network_label}.csv", header=0, index_col=0)

    labels = pd.read_csv(labels_dir + f"{dataset_label}_labels_{network_label}.csv", header=None)   
    
    ####################### Prepare data #######################################################

    module_labels = metrics.columns.to_list()

    X = metrics.to_numpy()
    y = labels.to_numpy(dtype='int')
    
    ###################### Compute GAP-MINE ######################################################
    clf_results = Parallel(n_jobs=-1)(
        delayed(classifiers.baseline)(
            X[:, module], y[:, module], beta=0.5, false_annotations=fa, shortest_paths=sp, graph=g, random_state=0
            ) for module in tqdm(range(len(module_labels)))
        )
    
    clf_results = pd.DataFrame(clf_results)
    clf_results['target_module'] = module_labels

    ###################### Save Results ###########################################################
    clf_results.to_csv(directory['clf_results']+f'{dataset_label}{fa_label}{network_label}.csv', index=False)

  0%|          | 0/426 [00:00<?, ?it/s]

  0%|          | 0/426 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/204 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]

  0%|          | 0/300 [00:00<?, ?it/s]